In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
import json
import requests
import warnings
import logging
from typing import List, Dict, Optional

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

warnings.filterwarnings('ignore')


In [ ]:
# URL của trang Wikipedia chứa danh sách phim 2018
wiki_url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

# Đọc các bảng từ Wikipedia
logging.info("Đang đọc dữ liệu từ Wikipedia...")
try:
    df1 = pd.read_html(wiki_url, header=0)[2]  # Q1
    df2 = pd.read_html(wiki_url, header=0)[3]  # Q2
    df3 = pd.read_html(wiki_url, header=0)[4]  # Q3
    df4 = pd.read_html(wiki_url, header=0)[5]  # Q4
    
    # Kết hợp dữ liệu từ 4 quý
    df = pd.concat([df1, df2, df3, df4], ignore_index=True)
    
    logging.info(f"Đã tải thành công {len(df)} phim từ Wikipedia")
    logging.info("\nMẫu dữ liệu:")
    display(df.head())
    
except Exception as e:
    logging.error(f"Lỗi khi đọc dữ liệu từ Wikipedia: {str(e)}")
    raise


In [ ]:
# Khởi tạo TMDB API
tmdb = TMDb()
tmdb.api_key = ''  # Thêm API key của bạn vào đây

# Khởi tạo Movie API
from tmdbv3api import Movie
tmdb_movie = Movie()

def get_genre(movie_title: str) -> Optional[str]:
    """
    Lấy thông tin thể loại phim từ TMDB API
    
    Args:
        movie_title: Tên phim cần tìm
        
    Returns:
        String chứa các thể loại phim, hoặc None nếu không tìm thấy
    """
    try:
        result = tmdb_movie.search(movie_title)
        if result:
            movie_id = result[0].id
            response = requests.get(
                f'https://api.themoviedb.org/3/movie/{movie_id}',
                params={'api_key': tmdb.api_key}
            )
            data_json = response.json()
            
            if data_json.get('genres'):
                genres = [genre['name'] for genre in data_json['genres']]
                return " ".join(genres)
                
    except Exception as e:
        logging.error(f"Lỗi khi lấy thông tin thể loại cho {movie_title}: {str(e)}")
    
    return None

# Áp dụng cho toàn bộ dataset
logging.info("Đang lấy thông tin thể loại từ TMDB API...")
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

# Kiểm tra kết quả
logging.info("\nMẫu dữ liệu sau khi thêm thể loại:")
display(df.head())


In [ ]:
# Lọc và chuẩn bị dữ liệu
df_2018 = df[['Title', 'Cast and crew', 'genres']]

# Hàm xử lý director
def get_director(x: str) -> str:
    """
    Trích xuất tên đạo diễn từ chuỗi Cast and crew
    """
    try:
        if " (director)" in x:
            return x.split(" (director)")[0]
        elif " (directors)" in x:
            return x.split(" (directors)")[0]
        else:
            return x.split(" (director/screenplay)")[0]
    except Exception as e:
        logging.error(f"Lỗi khi xử lý director: {str(e)}")
        return "unknown"

# Hàm xử lý actors
def get_actor1(x: str) -> str:
    """Lấy actor đầu tiên"""
    try:
        return ((x.split("screenplay); ")[-1]).split(", ")[0])
    except Exception as e:
        logging.error(f"Lỗi khi xử lý actor 1: {str(e)}")
        return "unknown"

def get_actor2(x: str) -> str:
    """Lấy actor thứ hai"""
    try:
        actors = (x.split("screenplay); ")[-1]).split(", ")
        return actors[1] if len(actors) > 1 else np.NaN
    except Exception as e:
        logging.error(f"Lỗi khi xử lý actor 2: {str(e)}")
        return np.NaN

def get_actor3(x: str) -> str:
    """Lấy actor thứ ba"""
    try:
        actors = (x.split("screenplay); ")[-1]).split(", ")
        return actors[2] if len(actors) > 2 else np.NaN
    except Exception as e:
        logging.error(f"Lỗi khi xử lý actor 3: {str(e)}")
        return np.NaN

# Áp dụng các hàm xử lý
logging.info("Đang xử lý thông tin cast và crew...")
df_2018['director_name'] = df_2018['Cast and crew'].map(get_director)
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(get_actor1)
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(get_actor2)
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(get_actor3)

# Kiểm tra kết quả
logging.info("\nMẫu dữ liệu sau khi xử lý:")
display(df_2018.head())


In [ ]:
# Tạo DataFrame cuối cùng với các cột cần thiết
new_df18 = df_2018.loc[:, ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'Title']]

# Xử lý missing values
logging.info("\n=== MISSING VALUES CHECK ===")
display(new_df18.isna().sum())

# Thay thế giá trị NaN cho actors
new_df18['actor_2_name'] = new_df18['actor_2_name'].fillna('unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].fillna('unknown')

# Đổi tên cột và chuẩn hóa
new_df18 = new_df18.rename(columns={'Title': 'movie_title'})
new_df18['movie_title'] = new_df18['movie_title'].str.lower()

# Tạo cột combined features
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' ' + new_df18['actor_3_name'] + ' ' + new_df18['director_name'] + ' ' + new_df18['genres']

# Kiểm tra dữ liệu cuối cùng
logging.info("\n=== FINAL DATA CHECK ===")
logging.info(f"Shape: {new_df18.shape}")
display(new_df18.head())

# Test integration với một phim
print(f"Testing data integration for: {test_movie}")
integrated_data = integrate_data(test_movie)
print("\nIntegrated data:")
for key, value in integrated_data.items():
    if isinstance(value, str):
        print(f"{key}: {value[:100]}...")
    else:
        print(f"{key}: {value}")

# Tích hợp dữ liệu cho toàn bộ dataset
print("\nProcessing all movies...")
all_data = []
for title in tqdm(data_2017['movie_title']):
    movie_data = integrate_data(title)
    all_data.append(movie_data)

# Convert to DataFrame
integrated_df = pd.DataFrame(all_data)
print("\nIntegrated DataFrame shape:", integrated_df.shape)
display(integrated_df.head())


In [ ]:
# Kết hợp với dữ liệu cũ
try:
    # Load dữ liệu cũ
    old = pd.read_csv('../datasets/data.csv')
    logging.info("\n=== LOADING OLD DATA ===")
    logging.info(f"Old data shape: {old.shape}")
    
    # Kiểm tra và tạo cột combined features cho dữ liệu cũ nếu chưa có
    if 'comb' not in old.columns:
        old['comb'] = old['actor_1_name'] + ' ' + old['actor_2_name'] + ' ' + old['actor_3_name'] + ' ' + old['director_name'] + ' ' + old['genres']
    
    # Kết hợp dữ liệu
    final_data = pd.concat([old, new_df18], ignore_index=True)
    logging.info(f"\nCombined data shape: {final_data.shape}")
    
    # Xuất ra file
    output_file = '../datasets/new_data.csv'
    final_data.to_csv(output_file, index=False)
    logging.info(f"\nĐã xuất dữ liệu kết hợp ra file: {output_file}")
    
except FileNotFoundError:
    logging.warning("\nKhông tìm thấy file data.csv cũ")
    output_file = '../datasets/new_data.csv'
    new_df18.to_csv(output_file, index=False)
    logging.info(f"\nĐã xuất dữ liệu mới ra file: {output_file}")
except Exception as e:
    logging.error(f"\nLỗi khi xử lý dữ liệu cũ: {str(e)}")
    raise


In [ ]:
def clean_text(text):
    """
    Clean và normalize text
    """
    if pd.isna(text):
        return ''
    
    # Convert to string
    text = str(text)
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase
    text = text.lower().strip()
    
    return text

# Clean text columns
text_columns = ['movie_title', 'plot', 'overview']
for col in text_columns:
    if col in integrated_df.columns:
        integrated_df[col] = integrated_df[col].apply(clean_text)

print("Sample cleaned text data:")
for col in text_columns:
    if col in integrated_df.columns:
        print(f"\n{col}:")
        print(integrated_df[col].iloc[0])


In [ ]:
def validate_integrated_data(df):
    """
    Validate dữ liệu sau khi tích hợp
    """
    print("=== INTEGRATED DATA VALIDATION ===")
    
    # Check missing values
    missing = df.isnull().sum()
    print("\nMissing values after integration:")
    print(missing[missing > 0] if missing.any() else "No missing values")
    
    # Check data types
    print("\nData types:")
    print(df.dtypes)
    
    # Check unique values
    print("\nUnique values per column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
    
    # Check text data quality
    print("\nText data quality check:")
    for col in ['movie_title', 'plot', 'overview']:
        if col in df.columns:
            empty_count = df[df[col].str.len() == 0].shape[0] if df[col].dtype == object else 0
            print(f"{col}:")
            print(f"  - Empty strings: {empty_count}")
            print(f"  - Sample values: {df[col].sample(3).tolist()}")
    
    return True

validate_integrated_data(integrated_df)


In [ ]:
# Lưu dữ liệu đã xử lý
integrated_df.to_csv('data_2017_integrated.csv', index=False)
print("Data saved successfully!")
